In [20]:
import numpy as np
from numpy.random import uniform 
import math
import cmath
from numpy import *
import matplotlib.pyplot as plt

**generating SU(3) matrices**

In [21]:
#generare matrice hermitiana H
def generate_hermitean():
    H=np.zeros((3,3),np.complex_)
    for i in range(3):
        for j in range (0,3):
            H[i][j]= uniform(-1,1) + 1j*uniform(-1,1)
    H = (1/2)*(H+np.matrix.getH(H))
    return H

In [22]:
#taylor expansion e matrice unitaria come esponenziale di una hermitiana
def unitarization(H):
    M=np.zeros((3,3),np.complex_)
   
    for n in range(90):
        M+=(((1j*eps)**n)/(np.math.factorial(n))*np.linalg.matrix_power(H,n))
    detM=np.linalg.det(M)**(1/3)
    M=M/detM
    
    return M

In [23]:
def generate_pool(N_mat):
    
    MatList=np.zeros((N_mat*2,3,3),np.complex_)
    H=np.zeros((3,3),np.complex_)
    
    #y = []
    #x= range(Nmat*2)    
    
    for i in range(N_mat):
        H=generate_hermitean()
        MatList[i]=unitarization(H)
        MatList[N_mat+i]=np.matrix.getH(unitarization(H))
        #y.append(np.real(np.linalg.det(MatList[i])))
        #y.append(np.real(np.linalg.det(MatList[Nmat+i])))
        
    return MatList

**lattice generation function**
generate lattice and 4 3x3 identity matrices on each site

In [24]:
def create_lattice(Npoints,dim):
    points = np.empty( (Npoints,Npoints,Npoints,Npoints,dim,3,3), np.complex_ )
    
    #print(lattice)
    for t in range(Npoints):
        for x in range(Npoints):
            for y in range(Npoints):
                for z in range(Npoints):
                    for mu in range(dim):
                        points[t,x,y,z,mu]=np.identity(3,np.complex_)    
    print("finish lattice creation")  #debug
    return points

**coordinate functions**

In [25]:
#moving a coordinate up
def up(coordinate,direction):
    coord_up=coordinate
    up=np.zeros((dimensions),np.int_)
    up[direction]=1
    #print(up)
    coord_up=(coordinate + up)%N_lat
    #print(coordinate)
    return coord_up
#moving a coordinate down
def down(coordinate,direction):
    coord_down=coordinate
    down=np.zeros((dimensions),np.int_)
    
    down[direction]=1
    coord_down=(coordinate-down)%N_lat
    return coord_down

def getlink(lattice,point,mu):
    link=lattice[point[0]%N_lat,point[1]%N_lat,point[2]%N_lat,point[3]%N_lat,mu]
    return link

**staple sum not improved**

In [26]:
def plaquette(lattice,point,mu,nu):
    staple = 0
    
    U_mu=lattice[point[0]%N_lat,point[1]%N_lat,point[2]%N_lat,point[3]%N_lat,mu]
    #plaquette points
    point_umu= up(point,mu)
    point_unu= up(point,nu)
    point_dnu=down(point,nu)
    point_umu_dnu=down(point_umu,nu)
    
    #point_nu=up(point,nu)
    #print(point)
    #print(point_umu)
    #print(point_unu)
    #print(point_dnu)
    #print(point_umu_dnu)
    #staple "up square"
    U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
    U_dag_mu_upnu=np.matrix.getH(lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],mu])
    U_dag_nu=np.matrix.getH(lattice[point[0],point[1],point[2],point[3],nu])
    #staple "down square"
    U_nu_downnu = lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],nu]
    U_dag_nu_downnu = np.matrix.getH(lattice[point_dnu[0],point_dnu[1],point_dnu[2],point_dnu[3],mu])
    U_dag_mu_upmudownnu = np.matrix.getH(lattice[point_umu_dnu[0],point_umu_dnu[1],point_umu_dnu[2],point_umu_dnu[3],mu] )
   
    staple += np.matmul( U_nu_upmu, np.matmul(U_dag_mu_upnu,U_dag_nu ))
    staple += np.matmul( U_nu_downnu, np.matmul(U_dag_nu_downnu,U_dag_mu_upmudownnu ))
    #print("finish plaquette")
    return staple / 3


**metropolis functions**

In [27]:
#metropolis step for one gauge link
def metropolis_link(lattice,point,mu,Xs):
    n_hits=10
    acceptedlink=0
    #get U
    U_old=getlink(lattice,point,mu) 
    staplesum=0
    #compute staple sum not improved
    for nu in range(dimensions):
        if nu != mu:
            staplesum+=plaquette(lattice,point,mu,nu)
    #do n_hits of metropolis update
    for i in range(n_hits):
        k=np.random.randint(0, Nmatrix)
        X=Xs[k]
        U_new= np.matmul(X,U_old)
    #calculate deltaS
        dS=deltaS(U_old,U_new,staplesum)
        if dS>0 and exp(-dS)<np.random.uniform(0,1):
            lattice[point[0],point[1],point[2],point[3],mu]=U_new
        else:
            acceptedlink+=1
    #print("finish metropolis link")
    return acceptedlink/n_hits

#metropolis step for the whole lattice
def metropolis_lattice(lattice,N_lat):
    acceptance=0
    for t in range(N_lat):
        for x in range(N_lat):
            for y in range(N_lat):
                for z in range(N_lat):
                    for mu in range(dimensions):
                        coord=[t,x,y,z]
                        acceptance+=metropolis_link(lattice,coord,mu,Xs)
    #print("finish metropolis lattice")
    return acceptance/dimensions/N_lat**dimensions

def deltaS(U_old,U_new,staplesum):
    beta=5.5
    return -(beta)*np.real(np.trace(np.matmul((U_new-U_old),staplesum)))


**wilson axa**

In [28]:
def wilson_axa(point,mu,nu):
    upstaple=0
    U_mu=lattice[point[0],point[1],point[2],point[3],mu]
    #points that will be used
    point_umu= up(point,mu)
    point_unu= up(point,nu)

    #"up square"
    U_nu_upmu=lattice[point_umu[0],point_umu[1],point_umu[2],point_umu[3],nu]
    U_dag_mu_upnu=np.matrix.getH(lattice[point_unu[0],point_unu[1],point_unu[2],point_unu[3],mu])
    U_dag_nu=np.matrix.getH(lattice[point[0],point[1],point[2],point[3],nu])
    upstaple = np.matmul(U_mu,np.matmul( U_nu_upmu, np.matmul(U_dag_mu_upnu,U_dag_nu )))
    #print("finish wilson link")
    return np.real(np.trace(upstaple))/3

#wilson axa loop average  over the whole lattice
def wilson_axa_lattice(lattice,N_lat):
    loopaxa_sum=0
    for t in range(N_lat):
        for x in range(N_lat):
            for y in range(N_lat):
                for z in range(N_lat):
                    for mu in range(dimensions):
                        for nu in range(mu):#??
                            point=[t,x,y,z]
                            loopaxa_sum+=wilson_axa(point,mu,nu)
    #print("finish wilson lattice")
    return loopaxa_sum/(N_lat**dimensions*6)

**main**

In [ ]:
N_lat = 8 #L/a
Nconf=100
Nmatrix=100
Ncor = 50
beta = 5.5
dimensions=4
eps=0.24
lat=create_lattice(N_lat,dimensions) #initialize lattice setting all links to identity
Xs=generate_pool(Nmatrix) #pool of random matrices to take the updates

def main_axa():
    wilson_axa=0 #set to zero and store the lattice avg of wilson axa loops
    for i in range(10*Ncor):#thermalize
        metropolis_lattice(lat,N_lat)
        print("thermalization number")
        print(i)
    print("FINISH TERMALIZE")
    for k in range(Nconf):
        for i in range(Ncor):
            metropolis_lattice(lat,N_lat)
        wilson_axa=wilson_axa_lattice(lat,N_lat)
        axa_avg+=wilson_axa
        print("finish conf", k)
    return axa_avg/Nconf

print(main_axa())

finish lattice creation
thermalization number
0
thermalization number
1
thermalization number
2
thermalization number
3
thermalization number
4
thermalization number
5
thermalization number
6
thermalization number
7
thermalization number
8
thermalization number
9
thermalization number
10
thermalization number
11
thermalization number
12
thermalization number
13
thermalization number
14
thermalization number
15
thermalization number
16
thermalization number
17
thermalization number
18
thermalization number
19
thermalization number
20
thermalization number
21
thermalization number
22
thermalization number
23
thermalization number
24
thermalization number
25
thermalization number
26
thermalization number
27
thermalization number
28
thermalization number
29
thermalization number
30
thermalization number
31
thermalization number
32
thermalization number
33
thermalization number
34
thermalization number
35
thermalization number
36
thermalization number
37
thermalization number
38
thermalizat

inizio @ 16.45
finish termalize @ 